In [60]:
import json

In [61]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [62]:
DATASET='val'
source_path = '/home/ml/cadencao/cnn-dailymail/cnn_dm/{}.source'.format(DATASET)
target_path = '/home/ml/cadencao/cnn-dailymail/cnn_dm/{}.target'.format(DATASET)

source = read_lines(source_path)
target = read_lines(target_path)

#### Sample

In [63]:
import spacy
import numpy as np
from tqdm import tqdm

nlp = spacy.load('en')

In [64]:
ID = 1

In [65]:
target[ID]

'The 20th MLS season begins this weekend . League has changed dramatically since its inception in 1996 . Some question whether rules regarding salary caps and transfers need to change .'

In [66]:
ents = []

sentence = target[ID]
for ent in nlp(sentence).to_json()['ents']:
    if ent['label'] != 'PERSON':
        ents.append(ent)
    else:
        name_parts = sentence[ent['start']: ent['end']].split()
        init_start = ent['start']
        for p in name_parts:
            ents.append({'start': init_start, 'end': init_start + len(p), 'label': 'PERSON'})
            init_start = init_start + len(p) + 1

In [67]:
ents

[{'start': 27, 'end': 39, 'label': 'DATE'},
 {'start': 97, 'end': 101, 'label': 'DATE'}]

In [68]:
sampled_id = np.random.randint(0, high=len(ents))
print(sampled_id)

1


In [69]:
target[ID][0: ents[sampled_id]['start']] + '###' + target[ID][ents[sampled_id]['end']:]

'The 20th MLS season begins this weekend . League has changed dramatically since its inception in ### . Some question whether rules regarding salary caps and transfers need to change .'

In [70]:
with open('cnndm_masked_{}.source'.format(DATASET), 'w', encoding='utf-8') as sf:
    with open('cnndm_masked_{}.target'.format(DATASET), 'w', encoding='utf-8') as tf:
        with open('cnndm_masked_{}.target.ent'.format(DATASET), 'w', encoding='utf-8') as etf:
            for s, t in tqdm(zip(source, target)):
                ents = []
                sentence = t
                for ent in nlp(sentence).to_json()['ents']:
                    if ent['label'] != 'PERSON':
                        ents.append(ent)
                    else:
                        name_parts = sentence[ent['start']: ent['end']].split()
                        init_start = ent['start']
                        for p in name_parts:
                            ents.append({'start': init_start, 'end': init_start + len(p), 'label': 'PERSON'})
                            init_start = init_start + len(p) + 1

                ent_num = len(ents)
                if ent_num > 0:
                    sampled_id = np.random.randint(0, high=ent_num)
                    masked_tgt = '<s> ' + sentence[0: ents[sampled_id]['start']] + '###' + sentence[ents[sampled_id]['end']:]
                    masked_src = masked_tgt + ' <\s> ' + s

                    sf.write(masked_src + '\n')
                    tf.write('<s> ' + sentence + '\n')
                    etf.write('<s> ' + sentence[ents[sampled_id]['start']: ents[sampled_id]['end']] + '\n')

13368it [02:35, 86.15it/s]


In [71]:
# pass